In [ ]:
import numpy as np
import pysindy as ps
from pysindy.feature_library import PolynomialLibrary, FourierLibrary, CustomLibrary, ConcatLibrary

import matplotlib.pyplot as plt
from IPython.display import display, Latex

from generate_data import generate_population_data

In [ ]:
t = np.linspace(0, 100, 1001)
x0 = [50, 50]

u = lambda t : np.sin(t)

x, x_dot = generate_population_data(t, x0, u)

In [ ]:
model = ps.SINDy(
    feature_library=PolynomialLibrary(degree=2), # 2, 1
    # feature_library=FourierLibrary(n_frequencies=1), # 2, 1
    optimizer=ps.STLSQ(threshold=0.001), # 0.001, 0.01, 0.1
    feature_names=[f'x{i+1}' for i in range(len(x_dot[0]))]+['u']
    )
model.fit(x=x, x_dot=x_dot, t=t, u=u(t))
model.print()

In [ ]:
x_sim = model.simulate(x0=x0, t=np.concatenate((t, [t[-1]+t[1]])), u=u(np.concatenate((t, [t[-1]+t[1]]))))
mse_x1 = ((x_sim[:, 0] - x[:, 0])**2).mean()
mse_x2 = ((x_sim[:, 1] - x[:, 1])**2).mean()
print(f'Błąd średniokwadratowy x1: {mse_x1:.2f}, x2: {mse_x2:.2f}')

In [ ]:
plt.plot(t, x[:, 0])
plt.plot(t, x[:, 1])
plt.plot(t, x_sim[:, 0], "r--")
plt.plot(t, x_sim[:, 1], "k--")
plt.ylim(0, max(x[:, 0]*1.4))
plt.xlim(0, max(t))
plt.grid()
plt.legend(["Populacja ofiar", "Populacja drapieżników", "Symulacja modelu ofiar", "Symulacja modelu drapieżników"])
plt.xlabel("Czas [dni]")
ax1 = plt.gca()
ax2 = ax1.twinx()
ax2.spines['right'].set_color('green')
ax2.yaxis.label.set_color('green')
ax2.tick_params(axis='y', colors='green')
ax2.plot(t, u(t), 'g--', alpha=0.4)
ax2.set_ylabel("Sterowanie")
ax2.set_ylim(-4, 4)
ax1.set_ylabel("Liczba osobników")
ax2.set_ylabel("Sterowanie")
plt.show()